In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models.keyedvectors import KeyedVectors


ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (C:\Users\p_adi\anaconda3\lib\site-packages\keras\preprocessing\sequence.py)

In [ ]:
data = pd.read_csv("dataset.csv")


In [ ]:
# Load the GloVe model
glove_model = KeyedVectors.load_word2vec_format('glove.6B.100d.txt', binary=False)

# Tokenize the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['correct_words'])

# convert words to numerical inputs using GloVe word embeddings
correct_words = pad_sequences(tokenizer.texts_to_sequences(data['correct_words']))
misspelled_words = pad_sequences(tokenizer.texts_to_sequences(data['misspelled_words']))

# Create the embedding matrix
embedding_matrix = np.zeros((len(tokenizer.word_index)+1, 100))
for word, i in tokenizer.word_index.items():
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]


In [ ]:
from keras.layers import Embedding, LSTM, Dense, Input, TimeDistributed
from keras.models import Model

# Define the input layer
input_layer = Input(shape=(None,))

# Define the embedding layer
embedding_layer = Embedding(len(tokenizer.word_index)+1, 100, input_length=None,weights=[embedding_matrix], trainable=False)(input_layer)

# Define the LSTM layer
lstm_layer = LSTM(100, return_sequences=True)(embedding_layer)

# Define the output layer
output_layer = TimeDistributed(Dense(len(tokenizer.word_index)+1, activation='softmax'))(lstm_layer)

# Create the model
model = Model(input_layer, output_layer)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(correct_words, misspelled_words)


In [ ]:
# Encode the misspelled words
misspelled_words = pad_sequences(tokenizer.texts_to_sequences(["


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import numpy as np

# load the GloVe embeddings
embeddings_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Tokenize the correct and misspelled words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(data['correct_words'].values) + list(data['misspelled_words'].values))

# Convert the words to sequences of integers
correct_sequences = tokenizer.texts_to_sequences(data['correct_words'].values)
misspelled_sequences = tokenizer.texts


In [ ]:
# Create an embedding matrix with the GloVe embeddings
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# Use the embedding matrix as the weights for the embedding layer
embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)
encoder_embedding = embedding_layer(encoder_inputs)
decoder_embedding = embedding_layer(decoder_inputs)

# Define and compile the model
model = Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Fit the model on the data
model.fit([correct_sequences, misspelled_sequences], correct_sequences,
          batch_size=32, epochs=100, validation_split=0.2)


In [ ]:
# Make predictions on new misspelled words
new_misspelled_words = ["appel", "bananaa", "orangge"]
new_misspelled_sequences = tokenizer.texts_to_sequences(new_misspelled_words)
new_misspelled_sequences = pad_sequences(new_misspelled_sequences, maxlen=max_length)

# Define the encoder and decoder models for making predictions
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(100,))
decoder_state_input_c = Input(shape=(100,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Use the encoder and decoder models to make predictions
states_value = encoder_model.predict(new_misspelled_sequences)
predicted_words = []
for i in range(len(new_misspelled_sequences)):
    decoded_sentence = ""
    target_seq = np.zeros((1, max_length))
    target_seq[0, 0] = 1
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word[sampled_token_index]
        decoded_sentence += " " + sampled_word
        if (sampled_word == "end" or len(decoded_sentence.split()) >= max_length):
            stop_condition = True
        target_seq = np.zeros((1, max_length))
        target_seq[0, 0] = sampled_token_


In [3]:
from spellchecker import SpellChecker

spell = SpellChecker()

# list of words to check
words = ['carrrot', 'pear', 'potatoe', 'womman', 'pumbkin', 'morpidopesity', 'metholienate', 'chelesteral', 'hyperdipillemia', 'hyperlifichesmia', 'folliaded',
             'Dratactive', 'towastatin', 'chelesteral', 'patision', 'Oregozale', 'Vigran', 'Farxija']

correct_words = []
incorrect_words = []

for word in words:
    if spell.correction(word) == word:
        correct_words.append(word)
    else:
        incorrect_words.append(word)

print("Correct words:", correct_words)
print("Incorrect words:", incorrect_words)


Correct words: ['pear']
Incorrect words: ['carrrot', 'potatoe', 'womman', 'pumbkin', 'morpidopesity', 'metholienate', 'chelesteral', 'hyperdipillemia', 'hyperlifichesmia', 'folliaded', 'Dratactive', 'towastatin', 'chelesteral', 'patision', 'Oregozale', 'Vigran', 'Farxija']


In [4]:
from spellchecker import SpellChecker

spell = SpellChecker()

# list of words to check
words = ['carrrot', 'pear', 'potatoe', 'womman', 'pumbkin', 'morpidopesity', 'metholienate', 'chelesteral', 'hyperdipillemia', 'hyperlifichesmia', 'folliaded',
             'Dratactive', 'towastatin', 'chelesteral', 'patision', 'Oregozale', 'Vigran', 'Farxija']

correct_words, incorrect_words = spell.batch_correction(words)

print("Correct words:", correct_words)
print("Incorrect words:", incorrect_words)


AttributeError: 'SpellChecker' object has no attribute 'batch_correction'

In [10]:
from spellchecker import SpellChecker

spell = SpellChecker()

# list of words to check
words = ['morpidopesity', 'metholienate', 'chelesteral', 'hyperdipillemia', 'hyperlifichesmia', 'folliaded',
             'Dratactive', 'towastatin', 'chelesteral', 'patision', 'Oregozale', 'Vigran', 'Farxija']

correct_words = [word for word in words if spell.correction(word) == word]
incorrect_words = [word for word in words if spell.correction(word) != word]

print("Correct words:", correct_words)
print("Incorrect words:", incorrect_words)


Correct words: []
Incorrect words: ['morpidopesity', 'metholienate', 'chelesteral', 'hyperdipillemia', 'hyperlifichesmia', 'folliaded', 'Dratactive', 'towastatin', 'chelesteral', 'patision', 'Oregozale', 'Vigran', 'Farxija']


In [7]:
import pandas as pd
from spellchecker import SpellChecker

data = pd.read_csv('sample_data.csv')

spell = SpellChecker()

def check_spelling(text):
    words = text.split()
    correct_words = {word: 'c' for word in words if spell.correction(word) == word}
    incorrect_words = {word: 'inc' for word in words if spell.correction(word) != word}
    return {**correct_words, **incorrect_words}

data['coutput_column'] = data['input_text'].apply(lambda x: check_spelling(x))


In [2]:
# List of sequences
sequences = [[1, 2, 3], [4, 5], [6, 7, 8, 9]]

# Maximum length of the sequences
max_len = 4

# Initialize a list to store the padded sequences
padded_sequences = []

# Iterate through each sequence
for sequence in sequences:
    # Calculate the number of padding elements needed
    num_padding = max_len - len(sequence)
    
    # Add the padding elements to the beginning of the sequence
    new_sequence = [0]*num_padding + sequence
    
    # Append the padded sequence to the list
    padded_sequences.append(new_sequence)

print(padded_sequences)


[[0, 1, 2, 3], [0, 0, 4, 5], [6, 7, 8, 9]]


In [3]:
# Maximum length of the sequences
max_len = 4

# Initialize a list to store the padded sequences
padded_sequences = []

# Iterate through each sequence
for sequence in sequences:
    # Calculate the number of padding elements needed
    num_padding = max_len - len(sequence)
    
    # Add the padding elements to the end of the sequence
    new_sequence = sequence + [0]*num_padding
    
    # Append the padded sequence to the list
    padded_sequences.append(new_sequence)


In [4]:
padded_sequences

[[1, 2, 3, 0], [4, 5, 0, 0], [6, 7, 8, 9]]